In [26]:
import boto3
import pandas as pd
import numpy as np
import io
from sklearn.metrics import accuracy_score

In [27]:
runtime = boto3.Session().client('runtime.sagemaker')

In [28]:
def get_payload(df):
    csv = io.StringIO()
    df.to_csv(csv,index=None,header=None)
    return csv.getvalue().rstrip()

In [29]:
# endpoint reference (see the "HowToDeploy" notebook)
cntkdemo_endpoint = 'cntkdemo-poc-endpoint-201801240604'

In [30]:
payload = get_payload(pd.read_csv('eval_data.csv',header=None))

In [31]:
%%time
response = runtime.invoke_endpoint(EndpointName=cntkdemo_endpoint,
                                   ContentType='text/csv',
                                   Body=payload)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 388 ms


In [32]:
result=response['Body'].read().decode()

In [33]:
y_pred = pd.read_csv(io.StringIO(result),sep=',',header=None)

In [34]:
y_pred

,0,1
0,0.035794,0.964206
1,0.218938,0.781062
2,0.971616,0.028384
3,0.903757,0.096243
4,0.920006,0.079994
5,0.012261,0.987739
6,0.925330,0.074670
7,0.885981,0.114019
8,0.260527,0.739473
9,0.898557,0.101443


In [35]:
y_true = pd.read_csv('eval_pred.csv',header=None)

In [36]:
# accuracy_score requires inputs to be 1-D arrays
yy_true = [np.argmax(i) for i in y_true.values]
yy_pred = [np.argmax(i) for i in y_pred.values]

print (yy_true)
print (yy_pred)

# overfitting baby...
print (accuracy_score(yy_true, yy_pred))

[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
1.0


### That's all!

## How to remove the endpoint

In [37]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
sess.delete_endpoint(cntkdemo_endpoint)